In [1]:
import torch
from torch.utils.cpp_extension import load

if torch.cuda.is_available():
    device = torch.device("cuda")

In [2]:
ultmul = load(name='ultmul', sources=['ultMul_cuda.cpp', 'ultMul_cuda_kernel.cu'])

import ultmul as um
um.ultmul

<function ultmul.PyCapsule.ultmul>

In [3]:
import sklearn.metrics.pairwise as sk_dist
torch.manual_seed(11000)
test = torch.randn(10001,10).float() # number of samples have to smaller than sqrt(512) need to fix
euc_dist = torch.from_numpy(sk_dist.euclidean_distances(test.numpy())).to(device).float()
print(euc_dist.size())
euc_dist

torch.Size([10001, 10001])


tensor([[0.0000, 3.8651, 3.7960,  ..., 5.0696, 4.0650, 3.8771],
        [3.8651, 0.0000, 3.6884,  ..., 3.6742, 3.8784, 3.2865],
        [3.7960, 3.6884, 0.0000,  ..., 5.0290, 3.1813, 2.8613],
        ...,
        [5.0696, 3.6742, 5.0290,  ..., 0.0000, 4.4713, 5.3355],
        [4.0650, 3.8784, 3.1813,  ..., 4.4713, 0.0000, 3.6948],
        [3.8771, 3.2865, 2.8613,  ..., 5.3355, 3.6948, 0.0000]],
       device='cuda:0')

In [4]:
#euc_dist
import time
torch.cuda.synchronize()   
start = time.clock()
u_test = um.ultmul(euc_dist, euc_dist)
torch.cuda.synchronize() 
end = time.clock()
tu = end-start
print(tu)

1.149624000000001


In [6]:
start = time.clock()
uclb = um.clusterability(euc_dist)
end = time.clock()
tc = end-start
print(uclb[0])
print(tc)

tensor([0.0042])
49.772478


In [7]:
start = time.clock()
uclust_label = um.single_hclust(uclb[1], 8)
end = time.clock()
print(end-start)
print(uclust_label)

1.232604000000002
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')


In [8]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
X = euc_dist.to('cpu').numpy()
start = time.clock()
clustering = AgglomerativeClustering(n_clusters = 8, affinity = 'precomputed', linkage='single').fit(X)
end = time.clock()
print(end-start)
print(clustering.labels_)

2.205869
[0 0 0 ... 0 0 0]


In [9]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(clustering.labels_, uclust_label.to('cpu').numpy())

1.0

In [10]:
torch.cuda.synchronize()  
start = time.clock()
m_test = torch.matmul(euc_dist, euc_dist)
torch.cuda.synchronize()
end = time.clock()
tm = end-start
print(tm)

0.42320799999999537
